In [1]:
# 主任务：利用爬虫收集新闻信息，进行人物观点分析
# 本脚本任务：爬取信息并保存到数据库中
import re

import pandas
import requests
from bs4 import BeautifulSoup
from urllib import request
import chardet
from urllib import parse

In [2]:
# 利用百度搜索引擎按关键词搜索热点新闻
keyword = input("请输入关键词(如：成龙)：")
url_baidu = "http://www.baidu.com/s?tn=news&word=" + keyword
print(url_baidu)
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

请输入关键词(如：成龙)：成龙
http://www.baidu.com/s?tn=news&word=成龙


In [3]:
# 定义函数用于提取百度搜索页面的新闻网址和下一页网址

def get_url(url):
    
     # 提取搜索页面网址    
    search_result = requests.get(url, headers=headers).text
    search_result = search_result.replace("\n","")   # 删除换行符，因为换行符不好匹配

    # 首先定位搜索结果所在的区间，避免引入大量垃圾网址
    results_re = r'<div class="result" id="\d+">(.*)</div>'
    results = re.compile(results_re).findall(search_result)

    # 在区间内搜索网址
    url_news_re = r'<a href="(https?://\S*)"'
    url_news = re.compile(url_news_re).findall(str(results))
    #print(url_news)
    
    # 提取百度下一页按钮网址
    url_next_page_re = '<a href="(\S*)"\ class="n">下一页&gt;</a>'
    url_next_page_0 = re.compile(url_next_page_re).findall(str(results))
    url_next_page = 'http://www.baidu.com' + url_next_page_0[0]
    print(url_next_page)
    
    return url_news, url_next_page
    

In [4]:
# 打开搜索结果中的网址，爬取新闻信息

def extract_text_by_bs(doc_text):
    """提取html页面的所有文本信息。 """
    title, text = '', ''
    soup = BeautifulSoup(doc_text, 'lxml')    # lxml, html5lib
    
    
    try:
        for script in soup(["script", "style"]):
            script.extract()
    except Exception as error:
        print('error')
        pass
    else:
        try:
            # get text
            title = soup.title.string
        except Exception as error:
            print('error')
            pass
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
    return text


# # 调试
# headers = {
#     'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
# url = 'http://baijiahao.baidu.com/s?id=1642514781800233125&wfr=spider&for=pc'
# # url = "http://k.sina.com.cn/article_6079563105_16a5ec56101900l9q4.html"
# news_page = requests.get(url, headers=headers)
# article = extract_text_by_bs(news_page.text)
# print('_'*50)
# print(article)


In [13]:
# 设置访问页数
max_number_of_page = 10000

# 用于保存每个网页的新闻信息
news = []

for page_number in range(max_number_of_page):
    
    # 第一次，给定地址,第二次以后，都用下一页地址
    if page_number == 0:
        url_news, url_next_page = get_url(url_baidu)
    else:
        url_news, url_next_page = get_url(url_next_page)

    # 开始遍历url_news中的每个网址，提取网页内容 
    for url in url_news:
        
        # 临时网址无效，不需要登录
        if 'cache' in url:
            continue
        
        # 提取网页信息，并判断其编码方式
        news_page = requests.get(url, headers=headers)
 
        # 登录不上的网址，不继续进行解析
        if "503 Service Temporarily Unavailable" in news_page.text:
            continue
       
        # 如果不知道是什么编码，则放弃这条新闻，不是好办法，先凑合着用
        try:
            response = request.urlopen(url)
            charset = chardet.detect(response.read())     #对该html进行编码的获取
            coding_style = charset['encoding']
            # 如果不是utf_8编码，则放弃这条新闻，不是好办法，先凑合着用
            if coding_style != 'utf-8':
                print(f'编码为{coding_style},不是utf-8编码，暂时放弃收录！', end=' ')
                continue
        except:
            print('解码错误！', end=' ')
            continue
         
        try:
            news_page.encoding = "utf-8"
            text = extract_text_by_bs(news_page.text)
        except:
            text = []
            
        if text is not []:
            #print(text[:100])
            news.append(text)
            print(len(news), end=' ')

http://www.baidu.com/s?tn=news&word=%E6%88%90%E9%BE%99&x_bfe_rqs=03E80&x_bfe_tjscore=0.491112&tngroupname=organic_news&rsv_dl=news_b_pn&pn=10
1 2 3 4 5 编码为GB2312,不是utf-8编码，暂时放弃收录！ 解码错误！ 6 7 8 9 9
http://www.baidu.com/s?tn=news&word=%E6%88%90%E9%BE%99&x_bfe_rqs=03E80&x_bfe_tjscore=0.135461&tngroupname=organic_news&rsv_dl=news_b_pn&pn=20
10 编码为Windows-1254,不是utf-8编码，暂时放弃收录！ 11 编码为Windows-1254,不是utf-8编码，暂时放弃收录！ 12 13 编码为ascii,不是utf-8编码，暂时放弃收录！ 解码错误！ 编码为GB2312,不是utf-8编码，暂时放弃收录！ 14 15 15
http://www.baidu.com/s?tn=news&word=%E6%88%90%E9%BE%99&x_bfe_rqs=03E80&x_bfe_tjscore=0.128319&tngroupname=organic_news&rsv_dl=news_b_pn&pn=30
16 17 18 19 解码错误！ 20 21 22 编码为ascii,不是utf-8编码，暂时放弃收录！ 23 24 24
http://www.baidu.com/s?tn=news&word=%E6%88%90%E9%BE%99&x_bfe_rqs=03E80&x_bfe_tjscore=0.105778&tngroupname=organic_news&rsv_dl=news_b_pn&pn=40
编码为UTF-8-SIG,不是utf-8编码，暂时放弃收录！ 25 编码为GB2312,不是utf-8编码，暂时放弃收录！ 26 解码错误！ 编码为GB2312,不是utf-8编码，暂时放弃收录！ 编码为GB2312,不是utf-8编码，暂时放弃收录！ 编码为GB2312,不是utf-8编码，暂时放弃收录！ 27 

ConnectionError: HTTPConnectionPool(host='www.jccinema.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x120f4e748>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [14]:
# 将新闻信息保存到文件中
import pickle
pickle.dump(news, open("news.pkl","wb"))

In [15]:
news

['成龙（中国香港演员、导演）_百度百科 百度首页 网页 新闻 贴吧 知道 音乐 图片 视频 地图 文库 百科 进入词条全站搜索帮助 清除历史记录关闭 声明：百科词条人人可编辑，词条创建和修改均免费，绝不存在官方及代理商付费代编，请勿上当受骗。详情>> 首页 历史上的今天 百科冷知识 图解百科 分类 艺术 科学 自然 文化 地理 生活 社会 人物 经济 体育 历史 秒懂百科 懂啦 秒懂星课堂 秒懂大师说 秒懂看瓦特 秒懂五千年 秒懂全视界 特色百科 数字博物馆 非遗百科 百度数说 城市百科 恐龙百科 多肉百科 用户 蝌蚪团 燃梦计划 百科任务 百科商城 权威合作 合作模式 常见问题 联系方式 个人中心 成龙是一个多义词，请在下列义项上选择浏览（共8个义项） 展开 收起 添加义项 ▪中国香港演员、导演 ▪美国动漫《成龙历险记》中的角色 ▪广东省委党校教授、硕士生导师 ▪动画片《奇幻龙宝》角色 ▪KOZAY,GALI演唱歌曲 ▪Preme/Post Malone演唱歌曲 ▪政协乌当区第十届委员会副主席 ▪淮安市洪泽区朱坝街道三圩居委会党总支书记 相关视频 图集 成龙图册 0 送花 今天的鲜花送完了？ 绑定百度钱包 即可领取500朵鲜花 [chéng lóng] 成龙 编辑 锁定 讨论999 1954年4月7日 香港 中国 成龙（Jackie Chan），1954年4月7日出生于香港，祖籍安徽芜湖，中国香港男演员、导演、动作指导、制作人、编剧、歌手 [1] 。1971年以武师身份进入电影圈 [2] 。1978年以电影《蛇形刁手》、《醉拳》确立功夫喜剧的动作风格 [3] 。1980年自导自演的动作片《师弟出马》打破香港地区票房纪录 [4] 。1986年自导自演的动作片《警察故事》获得第5届香港电影金像奖最佳影片奖 [5] 。1989年获得大不列颠帝国勋章 [3] 。1992年凭借警匪片《警察故事3超级警察》获得第29届台湾电影金马奖最佳男主角奖。1993年凭借警匪片《重案组》蝉联台湾电影金马奖最佳男主角 [6] 。1995年凭借动作片《红番区》打入美国好莱坞，该片打破北美外语片票房纪录 [7-8] 。1998年主演的动作片《尖峰时刻》奠定其在好莱坞的地位 [9] 。2001年主演的动作片《尖峰时刻2》创下华人演员主演好莱坞电影的票房纪录 [10] 。2005年凭借警匪片《新

In [7]:
# 随机查看文本
print(news[300:320])

[]
